In [13]:
from main import *
from ai import *
import pandas as pd
import numpy as np

#### Test Solver

In [67]:
test2 = '.5.....7.9......6...21.75.8.4..69.3...8...4...3.42..5.3.12.86...9......1.8.....2.'
test3 = '8...74...1.........7.5.9.4....65.4..73.....296.8.......2.3.1.5.........8...9....2'
solve(test2)

('Solved',
 '853946172917582364462137598145869237628375419739421856371298645294653781586714923')

In [68]:
a=solve_technique(test3,"naked_twins")
display(a[1])

   8     569    2369 |   12     7      4   | 12359   1369   1356 
   1     4569  23469 |   28    2368   236  | 235789 36789   3567 
   23     7     236  |   5    12368    9   |  1238    4     136  
---------------------+---------------------+---------------------
   29     19    129  |   6      5     237  |   4     1378   137  
   7      3      5   |   14     14     8   |   6      2      9   
   6     149     8   |   27    239    237  |  1357   137    1357 
---------------------+---------------------+---------------------
   49     2     4679 |   3     468     1   |   79     5     467  
  3459  14569  134679|  247    246    2567 |  1379  13679    8   
  345   14568  13467 |   9     468    567  |  137    1367    2   



In [25]:
# import sys
# sys.setrecursionlimit(1500)

In [14]:
df = pd.read_csv('data/dataset.csv').drop('Unnamed: 0', axis=1)
df.head()

,Id,Sudoku,Solution,Level,People,Average-Time
0,0,8...74...1.........7.5.9.4....65.4..73.....296...,8591742361648325973725698412916574837354186296...,Diabolical,254,25
1,1,.5.....7.9......6...21.75.8.4..69.3...8...4......,8539461729175823644621375981458692376283754197...,Moderate,281,12
2,2,...45.2..3.........8....4..7...94.3.9..5.2.76....,6974532183248617955819274637186945329435128762...,Diabolical,265,22
3,3,9......31.3...8.....5.7.9..3...4...8.916..54.4...,9274568316341982758153729643527496187916835424...,Tough,279,19
4,4,9.......2784.1.5.....4.....2.53.1.7...7...8......,9615387427846125395234796182953814761479568236...,Moderate,301,14


In [15]:
df['Sudoku'][0]

'8...74...1.........7.5.9.4....65.4..73.....296.8.......2.3.1.5.........8...9....2'

In [16]:
#solve(df['Sudoku'][0])
solve_technique(df['Sudoku'][0],'single_position')[0]

'Not solved'

In [17]:
solve_technique(df['Sudoku'][0],'single_candidate')[0]

'Not solved'

In [18]:
solve_technique(df['Sudoku'][0],'naked_twins')[0]

'Not solved'

In [19]:
solve(df['Sudoku'][0])

('Solved',
 '859174236164832597372569841291657483735418629648293175926381754413725968587946312')

In [4]:
import json
a =solve(df['Sudoku'][0])[1]

In [9]:
"".join([value if len(value) == 1 else "." for value in a.values()])

'859174236164832597372569841291657483735418629648293175926381754413725968587946312'

In [50]:
# rows = 'ABCDEFGHI'
# cols = '123456789'
# boxes = [s+t for s in rows for t in cols]

# def display(values):
#     """
#     Display the values as a 2-D grid.
#     Input: The sudoku in dictionary form
#     Output: None
#     """
#     width = 1+max(len(values[s]) for s in boxes)
#     line = '+'.join(['-'*(width*3)]*3)
#     for r in rows:
#         print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
#                       for c in cols))
#         if r in 'CF': print(line)
#     print

In [11]:
display(a)

8 5 9 |1 7 4 |2 3 6 
1 6 4 |8 3 2 |5 9 7 
3 7 2 |5 6 9 |8 4 1 
------+------+------
2 9 1 |6 5 7 |4 8 3 
7 3 5 |4 1 8 |6 2 9 
6 4 8 |2 9 3 |1 7 5 
------+------+------
9 2 6 |3 8 1 |7 5 4 
4 1 3 |7 2 5 |9 6 8 
5 8 7 |9 4 6 |3 1 2 


In [60]:
df['Solved_single_position'] = df['Sudoku'].apply(lambda x: solve_technique(x, 'single_position')[0])

In [61]:
df['Solved_single_position'].value_counts()

Not solved    4813
Solved         216
Name: Solved_single_position, dtype: int64

In [14]:
df['Solved_single_candidate'] = df['Sudoku'].apply(lambda x: solve_technique(x, 'single_candidate')[0])

In [15]:
df['Solved_single_candidate'].value_counts()

Not solved    5029
Name: Solved_single_candidate, dtype: int64

In [51]:
df['Solved_naked_twins']= df['Sudoku'].apply(lambda x: solve_technique(x, 'naked_twins')[0])

In [52]:
df['Solved_naked_twins'].value_counts()

Not solved    3658
Solved        1371
Name: Solved_naked_twins, dtype: int64

In [62]:
df[(df['Solved_naked_twins']=='Solved')& (df['Solved_single_position']=='Not solved')]

,Id,Sudoku,Solution,Level,People,Average-Time,Solved_single_position,Solved_naked_twins
1,1,.5.....7.9......6...21.75.8.4..69.3...8...4......,8539461729175823644621375981458692376283754197...,Moderate,281,12,Not solved,Solved
4,4,9.......2784.1.5.....4.....2.53.1.7...7...8......,9615387427846125395234796182953814761479568236...,Moderate,301,14,Not solved,Solved
8,8,...7....8..2.8.65.5.8.1.7.4.7...8.6..............,6347251987124896535983167243791485622865374191...,Moderate,272,15,Not solved,Solved
11,11,42....137...2.........17..8....85..4.76...9..5...,4289561377132486596593172482917853643761249855...,Moderate,261,15,Not solved,Solved
12,12,.96..3.4.8.....1..4....6.829....4.....4.5.3......,2961835478754291364315769829583642717149523683...,Moderate,292,14,Not solved,Solved
15,15,.54....8.....4..7.79...8.....26..1....35..2......,1547639823289456717962183459726341588435712965...,Moderate,274,14,Not solved,Solved
18,18,..3.8.4...2.....7.94...52.88..69....4.......7....,1637824595289436719471652388126975434362581977...,Moderate,264,15,Not solved,Solved
19,19,3.8..2....7.9.1.........48..61.9.2.74.......57...,3584621792749815636193754825618942374831276957...,Moderate,285,16,Not solved,Solved
26,26,...2.1.....5...2.3.8.57..1.7....8..9..86..1..9...,6742319581958462732835796147523184694386971259...,Moderate,277,15,Not solved,Solved
27,27,...8.31......2.3...27......85.....7.6..4.2..5....,9658431274186273593271594688513962746734129852...,Gentle,287,14,Not solved,Solved


In [18]:
df['Solved']= df['Sudoku'].apply(lambda x: solve(x)[0])

In [19]:
df['Solved'].value_counts()

Solved    5029
Name: Solved, dtype: int64

In [20]:
df.head()

,Id,Sudoku,Solution,Level,People,Average-Time,Solved_single_position,Solved_single_candidate,Solved_naked_twins,Solved
0,0,8...74...1.........7.5.9.4....65.4..73.....296...,8591742361648325973725698412916574837354186296...,Diabolical,254,25,Not solved,Not solved,Not solved,Solved
1,1,.5.....7.9......6...21.75.8.4..69.3...8...4......,8539461729175823644621375981458692376283754197...,Moderate,281,12,Not solved,Not solved,Not solved,Solved
2,2,...45.2..3.........8....4..7...94.3.9..5.2.76....,6974532183248617955819274637186945329435128762...,Diabolical,265,22,Not solved,Not solved,Not solved,Solved
3,3,9......31.3...8.....5.7.9..3...4...8.916..54.4...,9274568316341982758153729643527496187916835424...,Tough,279,19,Not solved,Not solved,Not solved,Solved
4,4,9.......2784.1.5.....4.....2.53.1.7...7...8......,9615387427846125395234796182953814761479568236...,Moderate,301,14,Not solved,Not solved,Not solved,Solved


In [21]:
display(solve(df['Sudoku'][0])[1])

8 5 9 |1 7 4 |2 3 6 
1 6 4 |8 3 2 |5 9 7 
3 7 2 |5 6 9 |8 4 1 
------+------+------
2 9 1 |6 5 7 |4 8 3 
7 3 5 |4 1 8 |6 2 9 
6 4 8 |2 9 3 |1 7 5 
------+------+------
9 2 6 |3 8 1 |7 5 4 
4 1 3 |7 2 5 |9 6 8 
5 8 7 |9 4 6 |3 1 2 


In [22]:
display(solve(df['Sudoku'][1])[1])

8 5 3 |9 4 6 |1 7 2 
9 1 7 |5 8 2 |3 6 4 
4 6 2 |1 3 7 |5 9 8 
------+------+------
1 4 5 |8 6 9 |2 3 7 
6 2 8 |3 7 5 |4 1 9 
7 3 9 |4 2 1 |8 5 6 
------+------+------
3 7 1 |2 9 8 |6 4 5 
2 9 4 |6 5 3 |7 8 1 
5 8 6 |7 1 4 |9 2 3 


In [23]:
df['Sudoku'][1]

'.5.....7.9......6...21.75.8.4..69.3...8...4...3.42..5.3.12.86...9......1.8.....2.'

In [28]:
test='.5.....7.9......6...21.75.8.4..69.3...8...4...3.42..5.3.12.86...9......1.8.....2.'

In [30]:
solve(test)[1]

'853946172917582364462137598145869237628375419739421856371298645294653781586714923'

In [34]:
a=solve(test)[1]

In [35]:
display(a)

8 5 3 |9 4 6 |1 7 2 
9 1 7 |5 8 2 |3 6 4 
4 6 2 |1 3 7 |5 9 8 
------+------+------
1 4 5 |8 6 9 |2 3 7 
6 2 8 |3 7 5 |4 1 9 
7 3 9 |4 2 1 |8 5 6 
------+------+------
3 7 1 |2 9 8 |6 4 5 
2 9 4 |6 5 3 |7 8 1 
5 8 6 |7 1 4 |9 2 3 



In [65]:
df['Sudoku'][4]
#test2 = '.5.....7.9......6...21.75.8.4..69.3...8...4...3.42..5.3.12.86...9......1.8.....2.'
test2='9.......2784.1.5.....4.....2.53.1.7...7...8...3.2.79.1.....3.....2.6.1958.......4'

In [70]:
rows = 'ABCDEFGHI'
cols = '123456789'

boxes = [s+t for s in rows for t in cols]
row_units = [[s+t for s in r for t in cols] for r in rows]
column_units = [[s+t for s in rows for t in c] for c in cols]
square_units = [[s+t for s in rs for t in cs]for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
valuesg = dict(zip(boxes,["." if element == "." else element for element in test3]))             

In [73]:
values = solve_technique(test3,"naked_twins")[1]
for unit in unitlist:
    twins = [v for v in [values[box] for box in unit] if [values[box] for box in unit].count(v) == 2 and len(v) == 2]
    print(twins)

[]
[]
[]
[]
['14', '14']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['14', '14']
[]
[]
[]
[]


In [75]:
display(values)

   8     569    2369 |   12     7      4   | 12359   1369   1356 
   1     4569  23469 |   28    2368   236  | 235789 36789   3567 
   23     7     236  |   5    12368    9   |  1238    4     136  
---------------------+---------------------+---------------------
   29     19    129  |   6      5     237  |   4     1378   137  
   7      3      5   |   14     14     8   |   6      2      9   
   6     149     8   |   27    239    237  |  1357   137    1357 
---------------------+---------------------+---------------------
   49     2     4679 |   3     468     1   |   79     5     467  
  3459  14569  134679|  247    246    2567 |  1379  13679    8   
  345   14568  13467 |   9     468    567  |  137    1367    2   



In [124]:
parejas=[]
for box in boxes:
    if len(values[box]) is 2:
        parejas.append(box)
print(parejas)

['A4', 'B4', 'C1', 'D1', 'D2', 'E4', 'E5', 'F4', 'G1', 'G7']


In [95]:
for column in column_units:
    if 'A4' in column:
        print(column)

['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4']


In [3]:
# len(validator(valuesg))
solve(test2)

('Ivalid Sudoku, check these values:', [False, '2', 'I9'])

In [203]:
display(values)

. 5 5 |. . . |. 7 . 
9 . . |. . . |. 6 . 
. . 2 |1 . 7 |5 . 8 
------+------+------
. 4 . |. 6 9 |. 3 . 
. . 8 |. . . |4 . . 
. 3 . |4 2 . |. 5 . 
------+------+------
3 . 1 |2 . 8 |6 . . 
. 9 . |. . . |. . 1 
. 8 . |. . . |. 2 . 


In [59]:
# def checker(grid):
    
#     rows = 'ABCDEFGHI'
#     cols = '123456789'

#     boxes = [s+t for s in rows for t in cols]
#     values_grid = dict(zip(boxes, ["." if element == "." else element for element in grid]))
#     row_units = [[s+t for s in r for t in cols] for r in rows]
#     column_units = [[s+t for s in rows for t in c] for c in cols]
#     square_units = [[s+t for s in rs for t in cs]for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
#     unitlist = row_units + column_units + square_units
#     units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
#     peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
    
# #     print(values_grid.keys())
# #     print('hiiiiiiiii',row_units)
#     if 'A1' in row_units:
#         print(row_units)

# #     for a in values_grid.values():
# #         if a == '.':
# #             print('no')
# #         else:
# #             if a in values_grid[a]:
# #                 print('yes')


In [34]:
def split(element): 
    return [char for char in element]  

def transform_matrix(element):
  return np.reshape(split(element), (-1, 9))

In [48]:
transform_matrix(test_wrong)

array([['.', '5', '.', '.', '.', '.', '.', '7', '.'],
       ['9', '.', '.', '.', '.', '.', '.', '6', '.'],
       ['.', '.', '2', '1', '.', '7', '5', '.', '8'],
       ['.', '4', '.', '.', '6', '9', '.', '3', '.'],
       ['.', '.', '8', '.', '.', '.', '4', '.', '.'],
       ['.', '3', '.', '4', '2', '.', '.', '5', '.'],
       ['3', '.', '1', '2', '.', '8', '6', '.', '.'],
       ['.', '9', '.', '.', '.', '.', '.', '.', '1'],
       ['.', '8', '.', '.', '.', '.', '.', '2', '.']], dtype='<U1')

In [35]:
transform_matrix(test2)

array([['.', '5', '.', '.', '.', '.', '.', '7', '.'],
       ['9', '.', '.', '.', '.', '.', '.', '6', '.'],
       ['.', '.', '2', '1', '.', '7', '5', '.', '8'],
       ['.', '4', '.', '.', '6', '9', '.', '3', '.'],
       ['.', '.', '8', '.', '.', '.', '4', '.', '.'],
       ['.', '3', '.', '4', '2', '.', '.', '5', '.'],
       ['3', '.', '1', '2', '.', '8', '6', '.', '.'],
       ['.', '9', '.', '.', '.', '.', '.', '.', '1'],
       ['.', '8', '.', '.', '.', '.', '.', '2', '.']], dtype='<U1')